# Neural Network (Google Colab version)
With this Jupyter notebook we can execute several operation to create, load or test a funtional Neural Network.

This Notebook has been prepared to be used in Google Colab and, due to this, it is necessary to use a Google Drive account.

It is also necessary to upload training and validation data in your Google Drive (more or less 100 MB). The upload time can be high (a couple of hours), but in google colab you can use a powerful GPU that some people (like me) can not afford, thus decreasing the training time of the neural network.

## Google Drive access
To use this notebook in google colab, you need to use your Google Drive account. You have to give Google rights to see, read and write on your Drive. Run this cell and you can work with your Google Drive data.

Remember to put in "commonURL" the path to the directory where all the data is upload. I recommend this structure:

Main

| - data

    | - train
    | - validation
    | - test

| - Neural_Network.h5

| - This_Notebook.ipynb

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls "/content/drive/My Drive/TFM/dataset"

"""
    Remember to change this Path
"""
commonURL = "/content/drive/My Drive/"

test  train  validation


## The Neural Network Architecture

In this next cell you can see the architecture used in the Neural Network.

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras import backend as K
from keras.applications import vgg16
from keras.optimizers import Adam
from keras.layers import Input

# Data dimensions of our images.
img_width, img_height = 128, 72

# The architecture
model = Sequential()
model.add(Conv2D(128, (20,20), padding='same', kernel_regularizer=regularizers.l2(0.0001), input_shape = (img_height, img_width,3)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (10,5), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (10,5), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (5,5), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(Flatten())
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 72, 128, 128)      153728    
_________________________________________________________________
batch_normalization_13 (Batc (None, 72, 128, 128)      512       
_________________________________________________________________
activation_13 (Activation)   (None, 72, 128, 128)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 36, 64, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 36, 64, 64)        409664    
_________________________________________________________________
batch_normalization_14 (Batc (None, 36, 64, 64)        256       
_________________________________________________________________
activation_14 (Activation)   (None, 36, 64, 64)        0         
__________

## Training

In this section we can train a new neural network or continue training an existing one.


### Continue a training

If you want to continue the training of a neural network you must load the neural network like you can see in the next cell.

In [0]:
del model

In [0]:
from keras.models import load_model
model = load_model('Neural_Network.h5')

### Training configuration
In the next cells there are several configuration parameters like the batch size or the number of epochs the network is going to be trained.

In [0]:
# Training data ubication
train_data_dir = commonURL + 'dataset/train'
validation_data_dir = commonURL + 'dataset/validation'

# Training configuration
nb_train_samples = 45000
nb_validation_samples = 4700
batch_size = 124
epochs = 10

In this cell we will upload the "dataset / train" data. There are many images, so ImageDataGenerator will be useful for that and will not break the computer's RAM. It is advisable to have checked the upper cell and made sure that the path where this folder is located is the correct one.

In [12]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator( 
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2)

# this is the augmentation configuration we will use for validation
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(72,128),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(72,128),
    batch_size=batch_size,
    class_mode='categorical')

Found 42093 images belonging to 7 classes.
Found 4679 images belonging to 7 classes.


### The epochs
In this cell we will train our model.

If we had used the previous model = load_model(commonURL + 'our_Neural_Network.h5') we can continue training a saved neural network.

In [0]:
# Training
import time
start = time.time()
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
end = time.time()

model.save(commonURL + 'new_checkpoint.h5')

Epoch 1/10
339/339 [==============================] - 216s 639ms/step - loss: 0.0756 - acc: 0.9919 - val_loss: 0.4593 - val_acc: 0.9207
Epoch 2/10
339/339 [==============================] - 210s 620ms/step - loss: 0.0767 - acc: 0.9917 - val_loss: 0.1276 - val_acc: 0.9769
Epoch 3/10
339/339 [==============================] - 207s 611ms/step - loss: 0.0752 - acc: 0.9916 - val_loss: 0.0772 - val_acc: 0.9901
Epoch 4/10
339/339 [==============================] - 209s 615ms/step - loss: 0.0698 - acc: 0.9937 - val_loss: 0.0879 - val_acc: 0.9849
Epoch 5/10
339/339 [==============================] - 209s 617ms/step - loss: 0.0868 - acc: 0.9875 - val_loss: 0.0599 - val_acc: 0.9982
Epoch 6/10
339/339 [==============================] - 208s 613ms/step - loss: 0.0695 - acc: 0.9942 - val_loss: 0.0706 - val_acc: 0.9932
Epoch 7/10
339/339 [==============================] - 207s 611ms/step - loss: 0.0727 - acc: 0.9922 - val_loss: 0.0950 - val_acc: 0.9866
Epoch 8/10
339/339 [============================